# Script writing for 5311 data

In [1]:
import numpy as np
import pandas as pd
from siuba import *
import altair as alt
import altair_saver
from calitp import *
from plotnine import *
import intake

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [4]:
import data_prep

In [5]:
#df = data_prep.load_grantprojects()
vehicles = data_prep.load_vehiclesdata()
# agency_info = data_prep.load_agencyinfo()
# gtfs_status = data_prep.load_catalog_gtfs()

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/home/jovyan/data-analyses/5311/data_prep.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [7]:
vehicles.sample()

,agency,city,state,legacy_ntd_id,ntd_id,organization_type,reporter_type,primary_uza_population,agency_voms,vehicle_type,years_old:,_13_15,_16_20,_21_25,_26_30,_31_60,_60+,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,_0_9,_10_12,vehicle_groups
6761,City of McFarland,Mc Farland,CA,9R02-024,9R02-91110,"City, County or Local Government Unit or Depar...",Rural Reporter,0,1,Bus,NaN,1,0,0,0,0,0,2,8.5,0.0,1,0,Bus


In [3]:
agency_info

,ntd_id,legacy_ntd_id,agency_name,reporter_acronym,doing_business_as,reporter_type,reporting_module,organization_type,reported_by_ntd_id,reported_by_name,subrecipient_type,fy_end_date,original_due_date,address_line_1,address_line_2,p_o__box,city,state,zip_code,zip_code_ext,region,url,fta_recipient_id,duns_number,service_area_sq_miles,service_area_pop,primary_uza,uza_name,tribal_area_name,population,density,sq_miles,voms_do,voms_pt,total_voms,volunteer_drivers,personal_vehicles,tam_tier,number_of_state_counties,number_of_counties_with_service,state_admin_funds_expended
884,90003,9003,San Francisco Bay Area Rapid Transit District,BART,NaN,Full Reporter,Urban,Independent Public Agency or Authority of Tran...,NaN,NaN,NaN,2020-06-30,2020-10-31,2150 Webster St,NaN,12688,Oakland,CA,94612.0,3012.0,9,http://www.bart.gov/,1957.0,47409107.0,80.0,858839.0,13.0,"San Francisco-Oakland, CA",NaN,3281212.0,6266.0,524.0,613.0,3.0,616.0,NaN,NaN,Tier I (Rail),NaN,NaN,NaN
885,90004,9004,Golden Empire Transit District,GET,NaN,Full Reporter,Urban,Independent Public Agency or Authority of Tran...,NaN,NaN,NaN,2020-06-30,2020-10-31,1830 Golden State Ave,NaN,NaN,Bakersfield,CA,93301.0,1012.0,9,http://www.getbus.org/,1695.0,75302935.0,111.0,500977.0,79.0,"Bakersfield, CA",NaN,523994.0,3785.0,138.0,91.0,NaN,91.0,NaN,NaN,Tier II,NaN,NaN,NaN
887,90006,9006,Santa Cruz Metropolitan Transit District,SCMTD,NaN,Full Reporter,Urban,Independent Public Agency or Authority of Tran...,NaN,NaN,NaN,2020-06-30,2020-10-31,110 Vernon St,NaN,NaN,Santa Cruz,CA,95060.0,2130.0,9,http://www.scmtd.com/,1675.0,92207398.0,446.0,274146.0,204.0,"Santa Cruz, CA",NaN,163703.0,2806.0,58.0,104.0,NaN,104.0,NaN,NaN,Tier II,NaN,NaN,NaN
888,90007,9007,City of Modesto,MAX,Modesto Area Express,Full Reporter,Urban,"City, County or Local Government Unit or Depar...",NaN,NaN,NaN,2020-06-30,2020-10-31,1010 10Th St Ste 2100,NaN,642,Modesto,CA,95354.0,847.0,9,http://www.modestoareaexpress.com/,1656.0,60125051.0,59.0,271314.0,105.0,"Modesto, CA",NaN,358172.0,3898.0,92.0,NaN,69.0,69.0,NaN,NaN,Tier II,NaN,NaN,NaN
889,90008,9008,City of Santa Monica,NaN,Big Blue Bus,Full Reporter,Urban,"City, County or Local Government Unit or Depar...",NaN,NaN,NaN,2020-06-30,2020-10-31,1334 5Th St,NaN,NaN,Santa Monica,CA,90401.0,3324.0,9,http://www.bigbluebus.com/,1664.0,833665896.0,59.0,855918.0,2.0,"Los Angeles-Long Beach-Anaheim, CA",NaN,12150996.0,6999.0,1736.0,162.0,31.0,193.0,NaN,NaN,Tier I (Fixed Route VOMS),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869,A0008,NaN,Kern Regional Transit,KT,NaN,Group Plan Sponsor,Asset,"City, County or Local Government Unit or Depar...",NaN,NaN,NaN,2020-06-30,2020-10-31,2700 M St Ste 400,Suite 400,NaN,Bakersfield,CA,93301.0,2375.0,9,http://kerntransit.org/,NaN,102541443.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A - Group Plan Sponsor,NaN,NaN,NaN
2870,A0009,NaN,Humboldt Transit Authority,HTA,NaN,Group Plan Sponsor,Asset,Independent Public Agency or Authority of Tran...,NaN,NaN,NaN,2020-06-30,2020-10-31,133 V St,NaN,NaN,Eureka,CA,95501.0,844.0,9,http://www.hta.org/,NaN,50506997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A - Group Plan Sponsor,NaN,NaN,NaN
2952,A0013,NaN,Metropolitan Transportation Commission,MTC,NaN,Group Plan Sponsor,Asset,"MPO, COG or Other Planning Agency",NaN,NaN,NaN,2020-06-30,2020-10-31,375 Beale St Ste 800,Suite 800,NaN,San Francisco,CA,94105.0,2179.0,9,http://www.mtc.ca.gov/,1655.0,74644501.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A - Group Plan Sponsor,NaN,NaN,NaN
2955,A0016,NaN,Redding Area Bus Authority,RABA,NaN,Group Plan Sponsor,Asset,Independent Public Agency or Authority of Tran...,NaN,NaN,NaN,2020-06-30,2020-10-31,777 Cypress Avenue,NaN,NaN,Redding,CA,96001.0,2718.0,9,http://www.rabaride.com/,NaN,556912103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A - Group P

## Read in Catalog

In [3]:
catalog = intake.open_catalog("catalog.yml")

In [4]:
gtfs_status = catalog.gtfs_status.read()

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [5]:
gtfs_status = to_snakecase(gtfs_status)

In [6]:
gtfs_status.head()

,name,itp_id,reporting_category,caltrans_district,gtfs_static_status,gtfs_realtime_status,missing_static,services_needing_alerts,services_needing_tripupdates_or_vehiclepositions
0,Long Beach Transit,170.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,AquaLink,"Galaxy Express, AquaLink, UCLA/Westwood Commut...","Galaxy Express, AquaLink, UCLA/Westwood Commut..."
1,City of Maywood,196.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,Maywood Express Shuttle,"Maywood Dial-A-Ride, Maywood Express Shuttle","Maywood Dial-A-Ride, Maywood Express Shuttle"
2,City of Cerritos,63.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,Cerritos on Wheels,"Cerritos on Wheels, Cerritos Dial-A-Ride","Cerritos on Wheels, Cerritos Dial-A-Ride"
3,City of La Puente,152.0,Other Public Transit,07 - Los Angeles,Static Incomplete,RT Incomplete,La Puente Link,"La Puente Link, La Puente Dial-A-Ride","La Puente Link, La Puente Dial-A-Ride"
4,City of Artesia,20.0,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,Artesia Transit,"Artesia Express Dial-A-Ride, Artesia Transit","Artesia Express Dial-A-Ride, Artesia Transit"


In [25]:
gtfs_status.name.nunique()

190

## Read in Vehicle Data

In [41]:
def load_vehiclesdata():
    vehicles_info =  pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020-Vehicles_1.xlsm',
                                   sheet_name = ['Age Distribution'])
    pd.set_option('display.max_columns', None)
    #cannot use to_snakecase because of integer column names
    vehicles_info = vehicles_info['Age Distribution']
    vehicles = (vehicles_info>>filter(_.State=='CA'))
    
    return vehicles

In [39]:
vehicles = load_vehiclesdata()

In [40]:
vehicles.sample(5)

,Agency,City,State,Legacy NTD ID,NTD ID,Organization Type,Reporter Type,Primary UZA Population,Agency VOMS,Vehicle Type,Years Old:,0,1,2,3,4,5,6,7,8,9,10,11,12,13-15,16-20,21-25,26-30,31-60,60+,Total Vehicles,Average Age Of Fleet (In Years),Average Lifetime Miles Per Vehicle
1072,Santa Cruz Metropolitan Transit District,Santa Cruz,CA,9006,90006,Independent Public Agency or Authority of Tran...,Full Reporter,163703,104,Cutaway,NaN,0,0,14,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,17,3.941176,59127.8824
2795,Imperial County Transportation Commission,El Centro,CA,9226,90226,Independent Public Agency or Authority of Tran...,Full Reporter,107672,28,Bus,NaN,0,0,0,0,0,6,0,0,10,0,0,0,0,0,0,0,0,0,0,16,6.875000,309605.6250
1942,Napa Valley Transportation Authority,Napa,CA,9088,90088,Independent Public Agency or Authority of Tran...,Full Reporter,83913,47,Automobiles (Service),NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,NaN,NaN
4457,Morongo Basin Transit Authority,Joshua Tree,CA,9R02-049,9R02-91090,Independent Public Agency or Authority of Tran...,Rural Reporter,0,13,Automobiles (Service),NaN,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,NaN,NaN
4156,City of Downey,Downey,CA,NaN,90263,"City, County or Local Government Unit or Depar...",Reduced Reporter,12150996,15,Cutaway,NaN,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,4,8.500000,78926.5000


* **note:** multiple entries for each agency

In [17]:
len(vehicles)

705

In [19]:
print(vehicles.Agency.nunique())
#print(vehicle_count.Agency.nunique())

218


## Read in Agency Information

In [26]:
def load_agencyinfo(): 
    agencies = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020_Agency_Information.xlsx',
                                      sheet_name=None),ignore_index=True)
    agencies = to_snakecase(agencies)
    agency_info = agencies>>filter(_.state=='CA')
    
    return agency_info

In [27]:
agency_info = load_agencyinfo()

In [30]:
agency_info.sample()

,ntd_id,legacy_ntd_id,agency_name,reporter_acronym,doing_business_as,reporter_type,reporting_module,organization_type,reported_by_ntd_id,reported_by_name,subrecipient_type,fy_end_date,original_due_date,address_line_1,address_line_2,p_o__box,city,state,zip_code,zip_code_ext,region,url,fta_recipient_id,duns_number,service_area_sq_miles,service_area_pop,primary_uza,uza_name,tribal_area_name,population,density,sq_miles,voms_do,voms_pt,total_voms,volunteer_drivers,personal_vehicles,tam_tier,number_of_state_counties,number_of_counties_with_service,state_admin_funds_expended
1029,90276,NaN,Los Angeles County Department of Public Works ...,LACDPW,NaN,Reduced Reporter,Urban,"City, County or Local Government Unit or Depar...",NaN,NaN,NaN,2020-06-30,2020-10-31,900 S. Fremont Avenue,NaN,NaN,Alhambra,CA,91803.0,NaN,9,www.dpw.lacounty.gov,5566.0,99446254.0,5.0,86883.0,2.0,"Los Angeles-Long Beach-Anaheim, CA",NaN,12150996.0,6999.0,1736.0,NaN,5.0,5.0,NaN,NaN,Tier II,NaN,NaN,NaN


## Read in Agency Revenue

In [85]:
agency_revenue = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020_revenue_sources.xlsx',
                                      sheet_name=None), ignore_index=True)

In [86]:
agency_revenue = to_snakecase(agency_revenue)

In [125]:
agency_revenue.sample(5)

,ntd_id,agency_name,reporter_type,subrecipient_type,reporting_module,funding_category,funds_expended_type,total_of_passenger_fares,park_and_ride_revenue__earned_only_,other_agency_revenue__earned_only_,...,funds_received_from_other_usdot_grant_programs,other_usdot_grants_description,other_federal_funds,other_federal_funds_description,contributed_services,voluntary_non_exchange_transactions,voluntary_non_exchange_transactions_description,sales_and_disposals_of_assets,transportation_development_credits,total
6597,40251,City of Pembroke Pines,Reduced Reporter,NaN,Urban,State Government,Funds Expended on Capital,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4711,3R06-30154,Greene Co. Transit Inc.,Rural Reporter,Rural General Public Transit,Rural,State Government,Funds Expended on Operations,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,215198
13902,6R03-60224,The Community Pantry,Rural Reporter,Rural General Public Transit,Rural,Local Government,Funds Expended on Operations,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
19473,90302,San Bernardino County Transportation Authority,Full Reporter,NaN,Urban,State Government,Funds Earned During Period,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
15054,7R02-70102,Decatur County Transportation Bus,Rural Reporter,Rural General Public Transit,Rural,Local Government,Funds Expended on Capital,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [115]:
agency_revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20446 entries, 0 to 20445
Data columns (total 93 columns):
 #   Column                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                --------------  -----  
 0   ntd_id                                                                                                                20446 non-null  object 
 1   agency_name                                                                                                           20446 non-null  object 
 2   reporter_type                                                                                                         20446 non-null  object 
 3   subrecipient_type                                                                                                     9618 non-null   object 
 4   repo

* no state information. have to merge with other dataframes to get the information we need.

## Read BlackCat Data

In [10]:
def load_grantprojects(): 
    df = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /Grant_Projects.xlsx', sheet_name=None), ignore_index=True)
    
    pd.set_option('display.max_columns', None)
    
    #a bit of cleaning
    df = to_snakecase(df)
    df = df.drop(columns=['project_closed_by', 'project_closed_date', 'project_closed_time'])
    
    #get just the 5311 agencies
    df = (df>>filter(_.funding_program.str.contains('5311')))
    
    return df

In [11]:
df = load_grantprojects()

In [12]:
df.sample()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status
13,2014,Section 5311,CA-18-X069 | 0015000075,2016,City of Ojai,BCG0000161,Purchase Replacement < 30 Ft Bus,111204,64BC17-00378,291536.0,291536.0,291536.0,0.0,0,Open


In [13]:
len(df)

798

In [14]:
df>>count(_.funding_program)

,funding_program,n
0,5311(f) Cont,41
1,5311(f) Round 2,27
2,CMAQ (FTA 5311),31
3,Section 5311,590
4,Section 5311(f),109


In [17]:
df_agency = df>> count(_.organization_name) >> arrange(-_.n)

In [18]:
print(len(df_agency.organization_name.unique()))

88


In [20]:
df_agency.head()

,organization_name,n
60,Mountain Area Regional Transit Authority,33
76,Sonoma County Transit,31
57,Modoc Transportation Agency,28
58,Monterey-Salinas Transit,28
87,Yuba-Sutter Transit Authority,28


In [21]:
sorted(df_agency.organization_name.unique())

['Alpine County Community Development',
 'Amador Transit',
 'Butte County Association of Governments/ Butte Regional Transit',
 'Calaveras County Public Works',
 'Calaveras Transit Agency ',
 'City of Arcata',
 'City of Arvin',
 'City of Auburn',
 'City of California City',
 'City of Chowchilla ',
 'City of Corcoran - Corcoran Area Transit',
 'City of Dinuba ',
 'City of Dixon',
 'City of Escalon ',
 'City of Fairfield',
 'City of Guadalupe',
 'City of McFarland',
 'City of Needles',
 'City of Ojai',
 'City of Porterville',
 'City of Ridgecrest',
 'City of Rio Vista',
 'City of Santa Maria',
 'City of Shafter',
 'City of Solvang',
 'City of Taft',
 'City of Tehachapi',
 'City of Visalia',
 'City of Wasco',
 'City of Woodlake',
 'Colusa County Transit Agency',
 'County Connection (Central Contra Costa Transit Authority)',
 'County of Los Angeles - Department of Public Works',
 'County of Mariposa',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Depar

* no obvious duplicates or multiple names for one agency

In [23]:
df >> group_by(_.organization_name) >> summarize(n=_.ali.nunique()) >> arrange(-_.n)

,organization_name,n
46,Humboldt Transit Authority,6
18,City of Ojai,5
58,Monterey-Salinas Transit,5
12,City of Dixon,4
54,Madera County,4
...,...,...
80,Town of Truckee,1
81,Transit Joint Powers Authority for Merced County,1
83,Tuolumne County Transit Agency (TCTA),1
84,Victor Valley Transit Authority,1


In [24]:
df >> group_by(_.organization_name) >> summarize(n=_.grant_number.nunique()) >> arrange(-_.n)

,organization_name,n
60,Mountain Area Regional Transit Authority,14
46,Humboldt Transit Authority,13
57,Modoc Transportation Agency,13
39,Eastern Sierra Transit Authority,12
58,Monterey-Salinas Transit,12
...,...,...
50,Klamath Trinity Non-Emergency Transportation​,3
63,Palo Verde Valley Transit Agency,3
14,City of Fairfield,2
53,Livermore Amador Valley Transit Authority,2


## Looking at Agency Names and NTD IDs

In [4]:
agency_crosswalk = agency_info[['agency_name','ntd_id']]

In [5]:
agency_crosswalk>>count(_.agency_name)>>arrange(-_.n)>>filter(_.n>1)

,agency_name,n
146,Humboldt Transit Authority,2
148,Kern Regional Transit,2
165,Los Angeles County Metropolitan Transportation...,2
170,Metropolitan Transportation Commission,2
205,San Luis Obispo Regional Transit Authority,2


In [6]:
agency_crosswalk>>filter(_.agency_name=='Humboldt Transit Authority')

,agency_name,ntd_id
2793,Humboldt Transit Authority,9R02-91036
2870,Humboldt Transit Authority,A0009


In [10]:
#checking to see if the NTD ID is the same in the Vehicles dataset
(vehicles>>filter(_.Agency=='Humboldt Transit Authority'))

,Agency,City,State,Legacy NTD ID,NTD ID,Organization Type,Reporter Type,Primary UZA Population,Agency VOMS,Vehicle Type,Years Old:,0,1,2,3,4,5,6,7,8,9,10,11,12,13-15,16-20,21-25,26-30,31-60,60+,Total Vehicles,Average Age Of Fleet (In Years),Average Lifetime Miles Per Vehicle
4296,Humboldt Transit Authority,Eureka,CA,9R02-016,9R02-91036,Independent Public Agency or Authority of Tran...,Rural Reporter,0,14,Trucks and other Rubber Tire Vehicles (Service),NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN
4297,Humboldt Transit Authority,Eureka,CA,9R02-016,9R02-91036,Independent Public Agency or Authority of Tran...,Rural Reporter,0,14,Cutaway,NaN,0,0,0,2,1,5,0,1,1,0,0,3,0,0,0,0,0,0,0,13,6.384615,0.0
4298,Humboldt Transit Authority,Eureka,CA,9R02-016,9R02-91036,Independent Public Agency or Authority of Tran...,Rural Reporter,0,14,Bus,NaN,0,0,0,1,0,4,5,0,2,2,0,0,0,2,0,0,0,0,0,16,7.062500,0.0
4299,Humboldt Transit Authority,Eureka,CA,9R02-016,9R02-91036,Independent Public Agency or Authority of Tran...,Rural Reporter,0,14,Automobiles (Service),NaN,0,0,0,0,0,0,1,0,1,1,0,0,1,2,1,0,0,0,0,7,NaN,NaN


## Misc.